### Задание 1

* Для каждого слова подсчитайте процент документов, в которых содержится это слово. 
* Сформируйте dataframe doc_frequency. 
* Процент документов вычисляется по формуле:

$$
d = \frac{m}{n} \times 100
$$

* где $m$ - количество документов, в которых встретилось это слово, 
* а $n$ - общее количество документов. 


* Значение в процентах округлите до целых.
* Посмотрите, какие слова оказались в топе - это предлоги и частицы. 
* Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". 
* Это упростит анализ текста.

In [ ]:
# Импортируем все необходимые библиотеки:

import re
import pandas as pd
import pymorphy2
import itertools  
import pandas as pd

In [10]:
# Загрузим данные:

text_df = pd.read_csv("content_description.csv", sep='\t')
text_df.head()

,content,description
0,https://www.ivi.ru/watch/157318/description,"Лучший подарок, который только можно было прид..."
1,https://www.ivi.ru/watch/98336/description,Через какие трудности приходится проходить Сан...
2,https://www.ivi.ru/watch/183533/description,Миловидный Давид - позор для своего отца. Не в...
3,https://www.ivi.ru/watch/157319/description,Экранизация сатирического бестселлера Стивена ...
4,https://www.ivi.ru/watch/51342/description,«Леди удача» – авантюрная романтическая комеди...


In [2]:
# разбиваем ткест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)  # кол-во всех документов
doc_ids = []  # айдишники документов
tokens = []  # наши сырые обработанные слова
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()
word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


In [3]:
doc_frequency = pd.DataFrame()
doc_frequency = word_count_df.groupby('word')['doc_id']\
    .count()\
    .reset_index(name='doc_freq')\
    .sort_values(by='doc_freq', ascending=False)

doc_frequency.doc_freq /= float(doc_count) # формула для расчета процента документов
doc_frequency.head(10)

,word,doc_freq
171,и,1.000
41,в,1.000
269,на,0.875
468,с,0.875
323,он,0.750
480,свой,0.750
618,что,0.750
326,онлайн,0.750
283,не,0.750
90,год,0.625


In [4]:
# 1-й вариант - чтобы удалить предлоги и частицы из строки, уберем все слова, длина которых меньше 2:

doc_frequency_2 = doc_frequency[doc_frequency['word'].str.len() > 2]
doc_frequency_2.head(10)

,word,doc_freq
480,свой,0.750
618,что,0.750
326,онлайн,0.750
90,год,0.625
498,смотреть,0.625
470,сам,0.500
483,себя,0.500
195,как,0.500
75,всё,0.500
565,тот,0.500


In [5]:
# 2-й вариант - тобы удалить предлоги и частицы из строки, определим части речи, используя pymorphy2:

def pos(word, morth=pymorphy2.MorphAnalyzer()):
    """" Return a likely part of speech for the word """
    return morth.parse(word)[0].tag.POS # принимает слово (обязательно в нижнем регистре) и возвращает все возможные его разборы

word_list = doc_frequency['word'].tolist() # превращаем в список все слова из столбца 'word'
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
pos_list = [word for word in word_list if pos(word) not in functors_pos]

In [6]:
finish_list = []
for i in word_list:
    if i in pos_list:
        finish_list.append(True)
    else:
        finish_list.append(False)

In [7]:
doc_frequency_3 = doc_frequency[finish_list]
doc_frequency_3.head(10)

,word,doc_freq
323,он,0.750
480,свой,0.750
326,онлайн,0.750
90,год,0.625
498,смотреть,0.625
470,сам,0.500
483,себя,0.500
565,тот,0.500
471,самый,0.500
218,который,0.500


### Задание 2

* Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.
* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [8]:
morph = pymorphy2.MorphAnalyzer()

raw_text = text_df.description.values[7]
# регулярка - её нужно поправить
reg_expr = r'\s\b[А-Я]\w+\s[А-Я]\w+'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

 Джеймс Кокс
 Тэрон Эджертон
 Энсел Эльгорт
 Кевин Спейси
 Джо Хант
 Дин Картни
 Рона Левина


### Задание 3

* Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. 
* Пользуясь словарём, а также таблицей `word_count_df`, сформируйте таблицу c двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно:

* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` с ранее полученным датафреймом `word_count_df`, который содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [9]:
genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres = [[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))  # возвращает по одному элементу из первого итератора, потом из второго, 
                                                        # до тех пор, пока итераторы не кончатся
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])

doc_genres = genres_df.merge(word_count_df, how = 'inner', left_on = 'character_word', right_on = 'word')[['doc_id', 'genre']]
doc_to_genres = doc_genres.groupby(['doc_id']).agg(list)  # чтобы разные жанры добавились через запятую
doc_to_genres

,genre
doc_id,
0,"[детектив, детектив, детектив, триллер]"
1,[сказка]
2,"[мелодрама, мелодрама]"
3,[комедия]
4,"[комедия, комедия]"
6,"[триллер, триллер, триллер]"
